In [1]:
from room import Room
from player import Player
from world import World
from graph import Graph
from util import opposite_dirs, Stack, Queue

import random
from ast import literal_eval

In [2]:
# Load world
world = World()


# You may uncomment the smaller graphs for development and testing purposes.
# map_file = "maps/test_line.txt"
# map_file = "maps/test_cross.txt"
# map_file = "maps/test_loop.txt"
# map_file = "maps/test_loop_fork.txt"
map_file = "maps/main_maze.txt"

# Loads the map into a dictionary
room_graph = literal_eval(open(map_file, "r").read())
world.load_graph(room_graph)

# Print an ASCII map
world.print_rooms()

player = Player(world.starting_room)

# Fill this out with directions to walk
# traversal_path = ['n', 'n']
traversal_path = []
visited = set()

#####
#                                                                                                                                                           #
#                                                        434       497--366--361  334--384--435  476                                                        #
#                                                         |                   |    |              |                                                         #
#                                                         |                   |    |              |                                                         #
#                                              477--443  425            386--354--321  338  313  380--445--446                                              #
#                                                    |    |              |         |    |    |    |    |                                                    #
#                                             

In [3]:
def random_move():
    # Get global vars
    global traversal_path
    
    # Save previous room before moving
    prev_room = player.current_room.id

    # If there are unexplored dirs, move that way
    direction = random.choice(unexplored_dirs)
    traversal_path.append(direction)

    # Move that direction
    player.travel(direction)

    # Add new room to graph.rooms and visited
    graph.add_current_room(player.current_room.id, player.current_room.get_exits())
    visited.add(player.current_room.id)

    # Connect rooms in graph
    graph.rooms[prev_room][direction] = player.current_room.id
    graph.rooms[player.current_room.id][opposite_dirs[direction]] = prev_room

In [4]:
def find_nearest_unexplored_room():
    """
    Basically a breadth-first search
    """
    # Global vars
    global traversal_path
    
    # Create empty queue & enqueue starting room
    queue = Queue()
    queue.enqueue([player.current_room.id])
    
    # Create a new visited set
    bfs_visited = set()
    
    while queue.size() > 0:
        
        # dequeue the current room/path
        path = queue.dequeue()
        cur_room = path[-1]
        
        if cur_room not in bfs_visited:
            
            # Check if there is an unexplored exit
            if "?" in graph.get_neighbors(cur_room):
                # If so, return the path
                return path
            
            # Mark as visited
            bfs_visited.add(cur_room)
            
            # Add it's neighbors
            for neighbor in graph.get_neighbors(cur_room):
                # Copy to avoid reference error
                new_path = list(path)
                new_path.append(neighbor)
                queue.enqueue(new_path)

In [5]:
def path_to_directions(path):
    directions = []

    for i in range(len(path) - 1):
        # Get current and next room
        cur_room, next_room = path[i], path[i+1]
#         print(cur_room, next_room)

        # Iterate through keys, look for entry matching next_room
        for key in graph.rooms[cur_room]:
            if graph.rooms[cur_room][key] == next_room:
#                 print(key)
                directions.append(key)
                
    return directions

In [6]:
# Instantiate Graph
graph = Graph()

# Add the first room to graph and visited
graph.add_current_room(player.current_room.id, player.current_room.get_exits())
visited.add(player.current_room.id)

In [7]:
while len(visited) < 500:
    # Find unexplored directions
    unexplored_dirs = []
    for key in graph.rooms[player.current_room.id]:
        if graph.rooms[player.current_room.id][key] == "?":
            unexplored_dirs.append(key)
            
    # If there are unexplored dirs, make a random move
    if len(unexplored_dirs) > 0:
        random_move()
        
    # If there are no unexplored dirs, bfs
    else:
        path = find_nearest_unexplored_room()
        directions = path_to_directions(path)
        
        # Iterate through directions:
        for direction in directions:
            # add to traversal path
            traversal_path.append(direction)
            
            # move that direction
            player.travel(direction)

In [8]:
graph.rooms

{0: {'n': '?', 's': '?', 'w': 3, 'e': 1},
 8: {'n': 0, 'w': 16},
 16: {'e': 8},
 4: {'s': 0},
 1: {'n': 7, 's': '?', 'w': 0, 'e': 22},
 2: {'n': 1, 's': 5, 'e': 10},
 5: {'n': 2, 's': '?', 'w': 6},
 50: {'n': 5, 's': 66, 'e': 70},
 70: {'s': 116, 'w': 50, 'e': 87},
 116: {'n': 70, 's': 159},
 159: {'n': 116},
 87: {'s': 117, 'w': 70},
 117: {'n': 87, 's': 127, 'e': 170},
 170: {'n': 182, 'w': 117},
 182: {'s': 170, 'e': 211},
 211: {'s': 248, 'w': 182},
 248: {'n': 211, 's': 272},
 272: {'n': 248},
 127: {'n': 117, 's': 212, 'e': 173},
 173: {'s': 202, 'w': 127},
 202: {'n': 173, 's': 267, 'e': 249},
 249: {'w': 202},
 267: {'n': 202, 'e': 302},
 302: {'w': 267, 'e': 402},
 402: {'w': 302, 'e': 403},
 403: {'w': 402, 'e': 439},
 439: {'w': 403},
 212: {'n': 127, 's': 229},
 229: {'n': 212, 's': 237},
 237: {'n': 229, 'e': 370},
 370: {'w': 237},
 66: {'n': 50, 's': 96},
 96: {'n': 66, 's': 179},
 179: {'n': 96, 's': 181, 'e': 201},
 181: {'n': 179},
 201: {'s': 206, 'w': 179},
 206: {'

In [9]:
traversal_path

['s',
 'w',
 'e',
 'n',
 'n',
 's',
 'e',
 's',
 's',
 's',
 'e',
 's',
 's',
 'n',
 'n',
 'e',
 's',
 'e',
 'n',
 'e',
 's',
 's',
 'n',
 'n',
 'w',
 's',
 'w',
 's',
 'e',
 's',
 'e',
 'w',
 's',
 'e',
 'e',
 'e',
 'e',
 'w',
 'w',
 'w',
 'w',
 'n',
 'n',
 'w',
 's',
 's',
 's',
 'e',
 'w',
 'n',
 'n',
 'n',
 'n',
 'n',
 'w',
 'w',
 's',
 's',
 's',
 's',
 'n',
 'e',
 's',
 's',
 's',
 's',
 's',
 's',
 's',
 'n',
 'n',
 'w',
 's',
 'n',
 'e',
 'n',
 'n',
 'e',
 'e',
 'e',
 'n',
 'e',
 'e',
 'e',
 'w',
 's',
 's',
 'e',
 'w',
 's',
 's',
 'n',
 'n',
 'n',
 'n',
 'w',
 's',
 's',
 'n',
 'n',
 'w',
 's',
 'w',
 'w',
 's',
 's',
 's',
 'n',
 'n',
 'e',
 's',
 's',
 'n',
 'n',
 'e',
 's',
 's',
 'n',
 'n',
 'w',
 'w',
 'n',
 'w',
 'n',
 'w',
 's',
 's',
 'n',
 'n',
 'e',
 'n',
 'n',
 'w',
 'n',
 'n',
 'n',
 'n',
 'w',
 'w',
 'w',
 'e',
 's',
 's',
 's',
 'w',
 'e',
 's',
 'w',
 'e',
 's',
 'e',
 'n',
 'n',
 'n',
 'n',
 'n',
 'e',
 'n',
 'n',
 'w',
 's',
 'w',
 'e',
 'n',
 'n',
 's',
 'w'

In [10]:
player.current_room.id

146

In [11]:
len(graph.rooms)

500

In [12]:
len(traversal_path)

2390